# Graphs (Uber)
##### *Algorithms & Data Structures*

A rule looks like this: `A NE B`; this means point `A` is located northeast of point `B`. Another rule looks like this: `A SW C`; this means point `A` is southwest of `C`.

Given a list of rules, check if the sum of the rules validate. For example, the rules below do not validate, since `A` cannot be both north and south of `C`.

In [1]:
from IPython.display import display, HTML
display(HTML('<table><tr><td><img src="img/rules.png"/></td></tr></table>'))

""


### Solution

Solution body